In [5]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import numpy as np
import matplotlib.pyplot as plt

In [18]:
target_number = int(input("Enter a number between 0 and 3: "))  
target_state = format(target_number, '02b')  # Converts to binary string ('00', '01', '10', '11')
print(f"Target state: |{target_state}⟩")

def oracle(n, target_state):
    qc = QuantumCircuit(n)
    
    # Apply X gates to flip target state to |11⟩ before applying CZ
    for i in range(n):
        if target_state[i] == '0':
            qc.x(i)

    qc.cz(0, 1)  # Phase flip on |11⟩

    # Undo the X gates
    for i in range(n):
        if target_state[i] == '0':
            qc.x(i)

    return qc

n = 2
qc = QuantumCircuit(n, n)

# Step 1: Apply Hadamard to all qubits (superposition)
qc.h(range(n))

# Step 2: Apply Grover Iterations
iterations = int(np.pi / 4 * np.sqrt(2**n))

for _ in range(iterations):
    qc.append(oracle(n, target_state), range(n))  # Apply oracle

    # Apply the diffusion operator
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.cx(0, 1)  # Corrected diffusion operator for n=2
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))

# Step 3: Measurement
qc.measure(range(n), range(n))

# Simulate
simulator = AerSimulator()
transpiled_qc = transpile(qc, simulator)
job = simulator.run(transpiled_qc, shots=1024)
result = job.result()
counts = result.get_counts()

# Display results
print("Grover's Algorithm Results:")
print(counts)

# Convert binary states back to decimal (0-3)
converted_counts = {}
for state, count in counts.items():
    decimal_state = int(state, 2)  # Convert binary string to decimal
    converted_counts[decimal_state] = count

# Display converted results
print("Converted Results (0-3):")
print(converted_counts)

Target state: |100⟩
Grover's Algorithm Results:
{'01': 1024}
Converted Results (0-3):
{1: 1024}
